In [36]:
# 查看当前kernel下已安装的包  list packages
!pip list --format=columns

Package                       Version
----------------------------- ----------------------
absl-py                       1.3.0
addict                        2.4.0
aiohttp                       3.8.3
aiosignal                     1.3.1
alabaster                     0.7.12
albumentations                1.3.0
aliyun-python-sdk-core        2.13.36
aliyun-python-sdk-kms         2.16.0
antlr4-python3-runtime        4.8
anyio                         3.6.2
appdirs                       1.4.4
argon2-cffi                   21.3.0
argon2-cffi-bindings          21.2.0
astor                         0.8.1
async-timeout                 4.0.2
asynctest                     0.13.0
attrs                         22.1.0
audioread                     3.0.0
autopep8                      2.0.0
av                            10.0.0
Babel                         2.11.0
backcall                      0.2.0
beautifulsoup4                4.11.1
bitarray                      2.6.0
bleach                        5.0.1


In [13]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
from sklearn.cluster import KMeans

In [23]:
#欧式距离计算
def euclidDistance(x1, x2, sqrt_flag=True):
    res = np.sum((x1-x2)**2)
    if sqrt_flag:
        res = np.sqrt(res)
    return res
#初始化聚类中心
def init_k_node(data,k):
    data =list(data)
    return random.sample(data,k)
#讲各节点分配给聚类中心
def get_clusters(data,k_centroids):
    cluster_dict =dict() #用来存储每个类别对应的节点信息
    k=len(k_centroids) #可获取设定的聚类个数
    labels = []
    for node in data:
        cluster_idx=-1 #设定初始类别为-1
        min_dis=float("inf") #设定初始聚类中心
        for idx in range(k): #计算节点同每个初始聚类中心的距离
            centroid=k_centroids[idx]
            distance =euclidDistance(node,centroid)
            if distance < min_dis:
                min_dis =distance
                cluster_idx=idx
                
        #存储每个类别包含的样本
        labels.append(cluster_idx)
        if cluster_idx not in cluster_dict.keys():
            cluster_dict[cluster_idx] = []
        cluster_dict[cluster_idx].append(node)
    return labels, cluster_dict

# 重新计算聚类中心
def get_centroids(cluster_dict):
    new_k_centroids=[]
    for cluster_idx in cluster_dict.keys():
        new_centroid = np.mean(cluster_dict[cluster_idx],axis=0)#每个类别中的均值
        new_k_centroids.append(new_centroid)
    return new_k_centroids
#计算各类间方差
def get_variance(centroids,cluster_dict):
    sum=0.0 #初始化均方误差为0
    for cluster_idx  in cluster_dict.keys():
        centroid = centroids[cluster_idx] #获取聚类中心
        distance= 0.0
        for node  in  cluster_dict[cluster_idx]:
            distance += euclidDistance(node,centroid)
        sum+=distance
    return sum
def kmeans(data, k):
    centroids=init_k_node(data,k) #获取初始聚类中心
    _, cluster_dict=get_clusters(data,centroids) #初始分类
    new_var=get_variance(centroids,cluster_dict) #计算初始聚类均方差
    old_var=1
    
    #设定条件，当两次聚类得误差小于某个值时，说明聚类基本稳定：
    while abs(new_var-old_var)>=0.00001:
        centroids =get_centroids(cluster_dict)
        labels, cluster_dict=get_clusters(data,centroids)
        old_var=new_var
        new_var=get_variance(centroids,cluster_dict)
        
    return labels, centroids,cluster_dict

print("以上为自定义K-means聚类代码")

以上为自定义K-means聚类代码


In [8]:
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.datasets import load_wine
import pandas as pd

In [9]:
#加载数据
wine = load_wine()
df = pd.DataFrame(wine.data, columns=wine.feature_names);
df['target'] = pd.Series(wine.target);
df

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline,target
0,14.23,1.71,2.43,15.6,127.0,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065.0,0
1,13.20,1.78,2.14,11.2,100.0,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050.0,0
2,13.16,2.36,2.67,18.6,101.0,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185.0,0
3,14.37,1.95,2.50,16.8,113.0,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480.0,0
4,13.24,2.59,2.87,21.0,118.0,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173,13.71,5.65,2.45,20.5,95.0,1.68,0.61,0.52,1.06,7.70,0.64,1.74,740.0,2
174,13.40,3.91,2.48,23.0,102.0,1.80,0.75,0.43,1.41,7.30,0.70,1.56,750.0,2
175,13.27,4.28,2.26,20.0,120.0,1.59,0.69,0.43,1.35,10.20,0.59,1.56,835.0,2
176,13.17,2.59,2.37,20.0,120.0,1.65,0.68,0.53,1.46,9.30,0.60,1.62,840.0,2


In [11]:
#标准化
scaler = StandardScaler()
x = scaler.fit_transform(wine.data)
y = wine.target
print(x)
print(y)

[[ 1.51861254 -0.5622498   0.23205254 ...  0.36217728  1.84791957
   1.01300893]
 [ 0.24628963 -0.49941338 -0.82799632 ...  0.40605066  1.1134493
   0.96524152]
 [ 0.19687903  0.02123125  1.10933436 ...  0.31830389  0.78858745
   1.39514818]
 ...
 [ 0.33275817  1.74474449 -0.38935541 ... -1.61212515 -1.48544548
   0.28057537]
 [ 0.20923168  0.22769377  0.01273209 ... -1.56825176 -1.40069891
   0.29649784]
 [ 1.39508604  1.58316512  1.36520822 ... -1.52437837 -1.42894777
  -0.59516041]]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]


In [24]:
# kmeans自定义实现
Labels, _, _=kmeans(x, 3)
x_v = x[Labels!=-1]
y_pred = np.array(Labels)
# print(x_v)
print(y_pred)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 2 1 1 1 1 1 1 1 1 1 1 1 0
 1 1 1 1 1 1 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 2 1 1 0 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]


In [18]:
# kmeans使用第三方库实现
km = KMeans(n_clusters=3,random_state=888)
result = km.fit_predict(x)
print(result)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 2 1 1 1 1 1 1 1 1 1 1 1 0
 1 1 1 1 1 1 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 2 1 1 0 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]


In [21]:
from sklearn import metrics
print('自定义kmeans准确率')
metrics.accuracy_score(y, y_pred)

自定义kmeans准确率


0.949438202247191

In [22]:
print('第三方库准确率')
metrics.accuracy_score(y, result)

第三方库准确率


0.9662921348314607